In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from sklearn_extra.cluster import KMedoids
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import itertools
from sklearn.manifold import TSNE

In [2]:
def CalculateCentroids(n, diheds):
    temp = pd.DataFrame(index = [f"phi:{n}_cos", f"phi:{n}_sin", f"psi:{n}_cos", f"psi:{n}_sin"], 
                        data = [np.cos(diheds[f"phi:{n}"]), np.sin(diheds[f"phi:{n}"]), 
                                np.cos(diheds[f"psi:{n}"]), np.sin(diheds[f"psi:{n}"])]).T

    dbscan = DBSCAN(eps=0.25)
    cluster_labels_DBSCAN = dbscan.fit(temp)
    
    if 1 in cluster_labels_DBSCAN.labels_:
        return temp
    return None

In [3]:
dihedsFL = [
    *[pd.read_csv(f"./Diheds/FL_Long/p53_ff14SB_FL_Rep{n}_WT_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad)
     for n in range(1, 4+1)],
    *[pd.read_csv(f"./Diheds/FL_Short/p53_ff14SB_FL_Rep{n}_WT_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad)
     for n in range(1, 10+1)],
    
    *[pd.read_csv(f"./Diheds/FL_Long/p53_ff14SB_FL_Rep{n}_Y220C_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad)
     for n in range(1, 4+1)],
    *[pd.read_csv(f"./Diheds/FL_Short/p53_ff14SB_FL_Rep{n}_Y220C_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad)
     for n in range(1, 10+1)],
    
    *[pd.read_csv(f"./Diheds/FL_Long/p53_ff14SB_FL_Rep{n}_WT_PK11000_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad)
     for n in range(1, 4+1)],
    *[pd.read_csv(f"./Diheds/FL_Short/p53_ff14SB_FL_Rep{n}_WT_PK11000_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad)
     for n in range(1, 10+1)],
    
    *[pd.read_csv(f"./Diheds/FL_Long/p53_ff14SB_FL_Rep{n}_Y220C_PK11000_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad)
     for n in range(1, 4+1)],
    *[pd.read_csv(f"./Diheds/FL_Short/p53_ff14SB_FL_Rep{n}_Y220C_PK11000_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad)
     for n in range(1, 10+1)]
]
dihedsFL = pd.concat(dihedsFL)
dihedsFL.reset_index(drop=True, inplace=True)
dihedsFL

,phi:2,psi:2,phi:3,psi:3,phi:4,psi:4,phi:5,psi:5,phi:6,psi:6,...,phi:388,psi:388,phi:389,psi:389,phi:390,psi:390,phi:391,psi:391,phi:392,psi:392
0,-0.897796,2.309013,-0.769931,-1.002334,-1.128252,0.057727,-1.239896,-0.615829,-1.154413,-0.279471,...,-1.148284,2.599572,-2.107616,3.112688,-1.066997,-0.368399,-1.343886,-0.081587,-1.393333,0.238208
1,-1.697987,2.668544,-1.179370,-0.717400,-1.203434,-0.455756,-1.503416,-0.398710,-1.247331,-0.346357,...,-1.332822,1.380534,-1.498267,-2.649138,-1.151303,-0.192850,-1.391308,-0.671467,-1.375831,0.362952
2,-1.654890,2.661432,-0.899049,-0.941321,-1.237678,-0.422783,-1.227240,-0.058161,-2.030394,0.076566,...,-1.115110,2.042191,-2.224759,3.121726,-0.923576,-0.575116,-1.190119,-0.237524,-1.955223,-0.053866
3,-1.347255,2.440712,-0.972930,-0.746669,-0.894099,-0.377384,-1.433074,-0.556558,-1.284215,-0.543209,...,-1.339971,1.762947,-1.549751,-2.694518,-1.158721,-0.416455,-1.673532,-0.158973,-1.747434,0.089480
4,-1.390917,2.790507,-1.225628,-0.747734,-0.779127,-0.661935,-1.362233,-0.438348,-1.749064,-0.182036,...,-0.937221,2.463476,-2.453291,-2.771789,-0.828244,-0.785744,-1.117663,-0.531378,-1.315428,0.594861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23939,-1.137218,2.781165,-1.191416,-0.891964,-1.037844,-0.225800,-1.598919,-0.374136,-1.304861,0.243644,...,-1.236875,2.911934,1.445010,-2.889298,-1.136098,-0.623933,-1.337348,0.117305,-2.569940,2.668608
23940,-1.321844,2.670015,-1.167683,-0.716667,-1.162433,-0.763229,-0.966834,-0.481992,-1.214693,-0.282597,...,-1.808440,3.040008,1.660545,-2.626354,-1.046232,-0.395963,-1.440799,-0.080125,-1.163382,2.897624
23941,-1.167683,2.672511,-1.075095,-0.744353,-1.257309,-0.244248,-1.341286,-0.426731,-1.411748,-0.148247,...,-1.079317,2.374053,2.636192,-2.925922,-1.056596,-0.596351,-1.269029,0.113569,-1.193001,2.350644
23942,-1.319162,2.784792,-0.857850,-1.016729,-1.174347,-0.620231,-1.067042,-0.480430,-1.333397,-0.147685,...,-2.163440,2.815052,1.566817,-2.748396,-1.162798,-0.323870,-1.209283,-0.515335,-1.265427,2.426182


In [4]:
dihedsFL_interesting = pd.concat([x for n in range(2, 392+1) if (x := CalculateCentroids(n, dihedsFL)) is not None], 
                                 axis=1)
dihedsFL_interesting

,phi:3_cos,phi:3_sin,psi:3_cos,psi:3_sin,phi:5_cos,phi:5_sin,psi:5_cos,psi:5_sin,phi:6_cos,phi:6_sin,...,psi:388_cos,psi:388_sin,phi:391_cos,phi:391_sin,psi:391_cos,psi:391_sin,phi:392_cos,phi:392_sin,psi:392_cos,psi:392_sin
0,0.717959,-0.696086,0.538337,-0.842730,0.324895,-0.945750,0.816295,-0.577635,0.404455,-0.914558,...,-0.856668,0.515868,0.224968,-0.974366,0.996674,-0.081497,0.176533,-0.984295,0.971762,0.235961
1,0.381508,-0.924366,0.753517,-0.657428,0.067329,-0.997731,0.921563,-0.388230,0.317854,-0.948140,...,0.189116,0.981955,0.178526,-0.983935,0.782910,-0.622135,0.193733,-0.981054,0.934853,0.355035
2,0.622355,-0.782735,0.588721,-0.808336,0.336837,-0.941563,0.998309,-0.058129,-0.443587,-0.896231,...,-0.454129,0.890936,0.371549,-0.928413,0.971924,-0.235296,-0.375027,-0.927014,0.998550,-0.053840
3,0.562880,-0.826539,0.733955,-0.679198,0.137287,-0.990531,0.849079,-0.528266,0.282675,-0.959216,...,-0.190970,0.981596,-0.102555,-0.994727,0.987390,-0.158305,-0.175721,-0.984440,0.995999,0.089360
4,0.338355,-0.941018,0.733232,-0.679979,0.207055,-0.978329,0.905454,-0.424444,-0.177325,-0.984152,...,-0.778756,0.627327,0.437784,-0.899080,0.862110,-0.506722,0.252601,-0.967570,0.828227,0.560393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23939,0.370345,-0.928894,0.627884,-0.778307,-0.028119,-0.999605,0.930824,-0.365468,0.262812,-0.964847,...,-0.973744,0.227646,0.231334,-0.972874,0.993128,0.117036,-0.841008,-0.541023,-0.890213,0.455545
23940,0.392284,-0.919844,0.753999,-0.656875,0.567908,-0.823092,0.886073,-0.463545,0.348624,-0.937263,...,-0.994845,0.101411,0.129632,-0.991562,0.996792,-0.080039,0.396236,-0.918149,-0.970387,0.241556
23941,0.475649,-0.879635,0.735526,-0.677496,0.227501,-0.973778,0.910324,-0.413897,0.158379,-0.987378,...,-0.719621,0.694367,0.297208,-0.954813,0.993558,0.113325,0.368873,-0.929480,-0.703171,0.711020
23942,0.654065,-0.756438,0.526150,-0.850392,0.482717,-0.875776,0.886796,-0.462160,0.235176,-0.971953,...,-0.947158,0.320768,0.353690,-0.935363,0.870128,-0.492826,0.300646,-0.953736,-0.754824,0.655927


In [5]:
temp = [[f"phi:{n}_cos", f"phi:{n}_sin", f"psi:{n}_cos", f"psi:{n}_sin"] for n in range(97, 288+1)]
DBD_cols = []
DBD_res = []
for x in temp:
    for y in x:
        DBD_cols.append(y)
        if y in dihedsFL_interesting:
            DBD_res.append(y)
dihedsFL_DBD_interesting = dihedsFL_interesting[DBD_res]
dihedsFL_DBD_interesting

,phi:97_cos,phi:97_sin,psi:97_cos,psi:97_sin,phi:102_cos,phi:102_sin,psi:102_cos,psi:102_sin,phi:105_cos,phi:105_sin,...,psi:238_cos,psi:238_sin,phi:240_cos,phi:240_sin,psi:240_cos,psi:240_sin,phi:247_cos,phi:247_sin,psi:247_cos,psi:247_sin
0,0.329394,-0.944193,-0.682213,0.731154,0.026456,-0.999650,-0.949377,0.314138,0.446654,0.894707,...,-0.780968,0.624571,0.459090,-0.888390,0.953862,-0.300245,0.698005,0.716092,0.749316,0.662212
1,0.408996,-0.912536,-0.739683,0.672956,0.203346,-0.979107,-0.752191,0.658945,0.274169,0.961682,...,-0.892832,0.450390,0.526982,-0.849876,0.999924,-0.012367,0.729860,0.683596,0.820441,0.571731
2,0.502453,-0.864605,-0.589244,0.807955,-0.149639,-0.988741,-0.847635,0.530579,0.379309,0.925270,...,-0.995979,0.089586,0.168875,-0.985638,0.584761,0.811206,0.618987,0.785401,0.836386,0.548141
3,0.197361,-0.980331,-0.484350,0.874874,0.155333,-0.987862,-0.743168,0.669105,0.444846,0.895607,...,-0.911193,0.411979,0.259937,-0.965626,0.999338,0.036389,0.742786,0.669529,0.736676,0.676246
4,0.686385,-0.727239,-0.333405,0.942784,-0.567061,-0.823676,-0.833305,0.552813,0.421408,0.906871,...,-0.789390,0.613893,0.386473,-0.922301,0.933408,0.358816,0.616627,0.787256,0.711243,0.702947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23939,0.191845,-0.981425,-0.676511,0.736433,0.213840,-0.976869,-0.747005,0.664818,0.662196,0.749330,...,-0.811950,0.583727,0.552333,-0.833624,0.927241,-0.374464,0.602740,0.797938,0.775215,0.631698
23940,0.101060,-0.994880,-0.783448,0.621457,0.485019,-0.874504,-0.829575,0.558395,0.349359,0.936989,...,-0.950134,0.311841,0.496732,-0.867904,0.998884,-0.047228,0.684254,0.729243,0.657295,0.753633
23941,0.519580,-0.854422,-0.891510,0.453001,0.434981,-0.900440,-0.915087,0.403256,0.313001,-0.949753,...,-0.917897,0.396819,0.533306,-0.845922,0.992450,-0.122649,0.745536,0.666465,0.787037,0.616905
23942,0.228344,-0.973580,-0.621872,0.783119,0.373566,-0.927604,-0.799495,0.600673,0.701534,-0.712636,...,-0.814058,0.580784,0.352190,-0.935929,0.928684,-0.370872,0.507895,0.861419,0.892763,0.450527


In [12]:
temp = [[f"phi:{n}", f"psi:{n}"] for n in range(97, 288+1)]
columns = []
for x in temp:
    for y in x:
        columns.append(y)

dihedsDBD = [
    *[pd.read_csv(f"./Diheds/DBD/p53_ff14SB_DBD_Rep{n}_WT_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad).set_axis(columns, axis=1)
     for n in range(1, 4+1)],
    
    *[pd.read_csv(f"./Diheds/DBD/p53_ff14SB_DBD_Rep{n}_Y220C_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad).set_axis(columns, axis=1)
     for n in range(1, 4+1)],
    
    *[pd.read_csv(f"./Diheds/DBD/p53_ff14SB_DBD_Rep{n}_WT_PK11000_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad).set_axis(columns, axis=1)
     for n in range(1, 4+1)],
    
    *[pd.read_csv(f"./Diheds/DBD/p53_ff14SB_DBD_Rep{n}_Y220C_PK11000_Dihed.dat", sep="\s+").drop("#Frame", axis=1).apply(np.deg2rad).set_axis(columns, axis=1)
     for n in range(1, 4+1)],
]
dihedsDBD = pd.concat(dihedsDBD)
dihedsDBD.reset_index(drop=True, inplace=True)
dihedsDBD

,phi:97,psi:97,phi:98,psi:98,phi:99,psi:99,phi:100,psi:100,phi:101,psi:101,...,phi:284,psi:284,phi:285,psi:285,phi:286,psi:286,phi:287,psi:287,phi:288,psi:288
0,-2.484841,2.544587,-1.092152,2.633636,-1.220043,2.320703,-1.537651,-0.349008,-1.182473,2.761621,...,-1.311772,-0.675720,-1.615588,-0.681029,-1.111169,-0.274991,-1.084123,-0.533223,-1.138417,-0.204710
1,-2.361212,2.405975,-1.466141,2.315521,-1.296125,2.816607,-1.286009,-0.323006,-1.144379,2.543905,...,-1.012195,-0.827469,-1.125439,-0.794060,-1.044030,-0.268226,-0.880943,-0.270488,-1.497871,-0.311892
2,-2.480544,2.209084,-1.083823,2.517114,-1.179710,2.232875,-1.601217,-0.237993,-1.021489,2.497301,...,-1.185552,-0.722882,-1.480106,-0.277869,-1.565951,-0.629181,-1.190859,-0.377087,-1.558179,-0.190761
3,-1.298834,2.447627,-1.505463,2.095936,-1.226192,2.377536,-2.543248,-0.123086,-0.933161,2.381270,...,-1.222847,-0.420572,-1.578526,-0.699289,-1.189136,-0.305136,-1.045046,-0.152121,-1.632605,-0.228619
4,-1.306958,2.291956,-1.002130,2.597722,-1.375145,2.130618,-2.373768,-0.359890,-1.159052,2.394278,...,-1.204223,-0.346022,-1.776782,-0.466125,-1.475725,-0.320495,-0.784245,-0.816421,-1.313311,0.461317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15979,-1.132375,2.698206,-1.250823,2.827116,-1.093801,2.267493,-2.849714,-0.295512,-0.793936,2.491618,...,-1.263774,-0.396888,-1.902165,-0.715911,-1.200973,-0.464362,-1.041272,-0.423611,-1.349436,-0.114879
15980,-1.442928,2.577066,-0.959647,2.701995,-1.025863,2.399304,-2.235809,-0.593539,-0.939664,2.317035,...,-1.371637,-0.729057,-1.131731,-0.577578,-1.354931,-0.363903,-1.253131,-0.470370,-1.375468,-0.224369
15981,-1.016281,2.407308,-1.101329,2.657300,-1.264559,2.612795,-2.501486,-0.081219,-1.046716,2.048207,...,-1.291694,-0.715372,-1.159143,-0.674493,-1.189419,-0.237063,-1.139011,-0.954273,-1.752386,0.231514
15982,-1.565426,2.701490,-1.283389,2.834046,-1.248368,2.160737,-1.693741,0.015748,-1.112599,2.105792,...,-1.075105,-1.120788,-0.987718,-0.818954,-1.049887,-0.371673,-1.289650,-0.376267,-1.420384,0.111031


In [13]:
dihedsDBD_interesting = pd.concat([x for n in range(97, 288+1) if (x := CalculateCentroids(n, dihedsDBD)) is not None], 
                                 axis=1)
dihedsDBD_interesting

,phi:97_cos,phi:97_sin,psi:97_cos,psi:97_sin,phi:100_cos,phi:100_sin,psi:100_cos,psi:100_sin,phi:101_cos,phi:101_sin,...,psi:286_cos,psi:286_sin,phi:287_cos,phi:287_sin,psi:287_cos,psi:287_sin,phi:288_cos,phi:288_sin,psi:288_cos,psi:288_sin
0,-0.791980,-0.610547,-0.827023,0.562169,0.033139,-0.999451,0.939712,-0.341966,0.378637,-0.925545,...,0.962428,-0.271538,0.467688,-0.883894,0.861174,-0.508311,0.419032,-0.907971,0.979120,-0.203283
1,-0.710646,-0.703550,-0.741416,0.671045,0.280953,-0.959721,0.948285,-0.317419,0.413611,-0.910454,...,0.964243,-0.265021,0.636424,-0.771339,0.963641,-0.267201,0.072860,-0.997342,0.951755,-0.306860
2,-0.789349,-0.613945,-0.595821,0.803117,-0.030416,-0.999537,0.971813,-0.235753,0.522097,-0.852886,...,0.808510,-0.588483,0.370862,-0.928688,0.929741,-0.368214,0.012617,-0.999920,0.981860,-0.189606
3,0.268622,-0.963246,-0.768716,0.639591,-0.826269,-0.563275,0.992434,-0.122775,0.595297,-0.803506,...,0.953806,-0.300423,0.501863,-0.864947,0.988452,-0.151535,-0.061770,-0.998090,0.973980,-0.226633
4,0.260788,-0.965396,-0.660256,0.751041,-0.719423,-0.694572,0.935935,-0.352172,0.400208,-0.916424,...,0.949080,-0.315036,0.707922,-0.706291,0.684833,-0.728700,0.254649,-0.967033,0.895467,0.445128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15979,0.424511,-0.905423,-0.903304,0.429001,-0.957705,-0.287752,0.956653,-0.291230,0.701044,-0.713118,...,0.894107,-0.447853,0.505123,-0.863048,0.911611,-0.411055,0.219557,-0.975600,0.993409,-0.114627
15980,0.127520,-0.991836,-0.844842,0.535016,-0.617069,-0.786909,0.828966,-0.559299,0.590059,-0.807360,...,0.934515,-0.355924,0.312350,-0.949967,0.891401,-0.453216,0.194089,-0.980984,0.974935,-0.222491
15981,0.526531,-0.850156,-0.742310,0.670056,-0.802032,-0.597281,0.996704,-0.081130,0.500417,-0.865784,...,0.972032,-0.234849,0.418493,-0.908220,0.578202,-0.815893,-0.180593,-0.983558,0.973320,0.229452
15982,0.005370,-0.999986,-0.904708,0.426032,-0.122635,-0.992452,0.999876,0.015747,0.442333,-0.896851,...,0.931721,-0.363175,0.277457,-0.960738,0.930043,-0.367451,0.149846,-0.988709,0.993842,0.110803


In [14]:
DBD_cols = []
for x in [[f"phi:{n}", f"psi:{n}"] for n in range(97, 288+1)]:
    for y in x:
        DBD_cols.append(y)
dihedsFL_DBD_AND_DBD = pd.concat([dihedsFL[DBD_cols], dihedsDBD])
dihedsFL_DBD_AND_DBD.reset_index(drop=True, inplace=True)
dihedsFL_DBD_AND_DBD

,phi:97,psi:97,phi:98,psi:98,phi:99,psi:99,phi:100,psi:100,phi:101,psi:101,...,phi:284,psi:284,phi:285,psi:285,phi:286,psi:286,phi:287,psi:287,phi:288,psi:288
0,-1.235135,2.321581,-1.285564,2.790450,-2.142903,2.494025,-1.889242,-0.132710,-1.268854,2.876721,...,-1.127134,-0.609050,-1.198786,-0.836212,-1.157647,-0.443462,-1.214016,-0.694774,-1.385612,-0.780351
1,-1.149442,2.403395,-1.196727,3.120628,-2.603286,2.386464,-1.744774,-0.377541,-1.027957,2.363504,...,-1.473744,-0.603097,-1.180337,-0.855986,-1.104249,-0.455203,-1.218599,-1.041934,-0.903039,-0.891436
2,-1.044363,2.200920,-1.266640,-3.106484,-2.757093,2.447770,-2.201607,-0.287065,-0.856833,2.447561,...,-1.214838,-0.632287,-1.200539,-0.750996,-1.538797,-0.139087,-1.288903,-1.042038,-0.894675,-0.730110
3,-1.372131,2.076416,-1.230941,2.515733,-1.888649,2.637880,-1.996599,-0.469809,-1.026051,2.173443,...,-1.255438,-0.734792,-1.243678,-0.724579,-1.292636,-0.400516,-1.144786,-0.776061,-1.213890,-0.777380
4,-0.814290,1.910710,-1.466326,2.391317,-1.590070,2.479362,-1.794621,0.092813,-1.338430,-3.134639,...,-1.106509,-1.075121,-1.080111,-0.810142,-1.171350,-0.347638,-1.122685,-0.856124,-1.064708,-0.791477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39923,-1.132375,2.698206,-1.250823,2.827116,-1.093801,2.267493,-2.849714,-0.295512,-0.793936,2.491618,...,-1.263774,-0.396888,-1.902165,-0.715911,-1.200973,-0.464362,-1.041272,-0.423611,-1.349436,-0.114879
39924,-1.442928,2.577066,-0.959647,2.701995,-1.025863,2.399304,-2.235809,-0.593539,-0.939664,2.317035,...,-1.371637,-0.729057,-1.131731,-0.577578,-1.354931,-0.363903,-1.253131,-0.470370,-1.375468,-0.224369
39925,-1.016281,2.407308,-1.101329,2.657300,-1.264559,2.612795,-2.501486,-0.081219,-1.046716,2.048207,...,-1.291694,-0.715372,-1.159143,-0.674493,-1.189419,-0.237063,-1.139011,-0.954273,-1.752386,0.231514
39926,-1.565426,2.701490,-1.283389,2.834046,-1.248368,2.160737,-1.693741,0.015748,-1.112599,2.105792,...,-1.075105,-1.120788,-0.987718,-0.818954,-1.049887,-0.371673,-1.289650,-0.376267,-1.420384,0.111031


In [15]:
dihedsFL_DBD_AND_DBD_interesting = pd.concat([x for n in range(97, 288+1) if (x := CalculateCentroids(n, dihedsFL_DBD_AND_DBD)) is not None], 
                                 axis=1)
dihedsFL_DBD_AND_DBD_interesting

,phi:97_cos,phi:97_sin,psi:97_cos,psi:97_sin,phi:100_cos,phi:100_sin,psi:100_cos,psi:100_sin,phi:101_cos,phi:101_sin,...,psi:286_cos,psi:286_sin,phi:287_cos,phi:287_sin,psi:287_cos,psi:287_sin,phi:288_cos,phi:288_sin,psi:288_cos,psi:288_sin
0,0.329394,-0.944193,-0.682213,0.731154,-0.313091,-0.949723,0.991207,-0.132320,0.297375,-0.954761,...,0.903272,-0.429069,0.349259,-0.937026,0.768199,-0.640212,0.184128,-0.982902,0.710667,-0.703529
1,0.408996,-0.912536,-0.739683,0.672956,-0.173102,-0.984904,0.929574,-0.368636,0.516569,-0.856245,...,0.898172,-0.439644,0.344961,-0.938617,0.504552,-0.863381,0.619227,-0.785212,0.628296,-0.777975
2,0.502453,-0.864605,-0.589244,0.807955,-0.589800,-0.807550,0.959079,-0.283138,0.654834,-0.755772,...,0.990343,-0.138639,0.278175,-0.960530,0.504461,-0.863434,0.625772,-0.780006,0.745101,-0.666951
3,0.197361,-0.980331,-0.484350,0.874874,-0.413052,-0.910707,0.891655,-0.452716,0.518200,-0.855259,...,0.920860,-0.389894,0.413241,-0.910622,0.713678,-0.700473,0.349377,-0.936982,0.712754,-0.701415
4,0.686385,-0.727239,-0.333405,0.942784,-0.221960,-0.975056,0.995696,0.092680,0.230281,-0.973124,...,0.940180,-0.340678,0.433264,-0.901267,0.655370,-0.755308,0.484759,-0.874648,0.702795,-0.711392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39923,0.424511,-0.905423,-0.903304,0.429001,-0.957705,-0.287752,0.956653,-0.291230,0.701044,-0.713118,...,0.894107,-0.447853,0.505123,-0.863048,0.911611,-0.411055,0.219557,-0.975600,0.993409,-0.114627
39924,0.127520,-0.991836,-0.844842,0.535016,-0.617069,-0.786909,0.828966,-0.559299,0.590059,-0.807360,...,0.934515,-0.355924,0.312350,-0.949967,0.891401,-0.453216,0.194089,-0.980984,0.974935,-0.222491
39925,0.526531,-0.850156,-0.742310,0.670056,-0.802032,-0.597281,0.996704,-0.081130,0.500417,-0.865784,...,0.972032,-0.234849,0.418493,-0.908220,0.578202,-0.815893,-0.180593,-0.983558,0.973320,0.229452
39926,0.005370,-0.999986,-0.904708,0.426032,-0.122635,-0.992452,0.999876,0.015747,0.442333,-0.896851,...,0.931721,-0.363175,0.277457,-0.960738,0.930043,-0.367451,0.149846,-0.988709,0.993842,0.110803


In [16]:
dihedsFL.to_csv("./Data/dihedsFL.csv")
dihedsFL_interesting.to_csv("./Data/dihedsFL_interesting.csv")
dihedsFL_DBD_interesting.to_csv("./Data/dihedsFL_DBD_interesting.csv")
dihedsDBD.to_csv("./Data/dihedsDBD.csv")
dihedsDBD_interesting.to_csv("./Data/dihedsDBD_interesting.csv")
dihedsFL_DBD_AND_DBD.to_csv("./Data/dihedsFL_DBD_AND_DBD.csv")
dihedsFL_DBD_AND_DBD_interesting.to_csv("./Data/dihedsFL_DBD_AND_DBD_interesting.csv")